In [2]:
import csv
import glob
import os
import random

from datasets import Dataset, DatasetDict

In [4]:
{'a': 1} | {'b': 2}

{'a': 1, 'b': 2}

In [17]:
data_dirs = [
    f"../cache_data/IBM-ArgQ-{n}kPairs/"
    for n in ['9.1', '14']
]


file_paths = [
    path
    for data_dir in data_dirs
    for path in glob.glob(f"{data_dir}/*.tsv")
]

data = []


for file_path in file_paths:
    file_name = os.path.basename(file_path)
    sentence = file_name.split('.')[0].replace('-', ' ').replace("(PRO)", "").replace("(CON)", "").strip()
    
    with open(file_path, 'r') as file:
        reader = csv.reader(file, delimiter="\t", quotechar='"')
        column_names = next(reader)
        
        data.extend([
            dict(
                zip(column_names, row)
            ) | dict(sentence=sentence)
            for row in reader
        ])

In [18]:
remove_columns = ["#id", "annotatorid"]

dataset = Dataset.from_list(data).remove_columns(remove_columns)

In [21]:
dataset[0]

{'label': 'a1',
 'a1': 'many institutions and businesses have money invested in fossil fuels, a ban would destroy the economy and those institutions.',
 'a2': 'in places with extreme weather conditions walking to public transportation or not being able to heat your house can be very dangerous.',
 'sentence': 'We should not ban fossil fuels'}

In [20]:
dataset.push_to_hub("diwank/IBMDebaterArgQ", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

def to_chatml(row):
    sentence = row["sentence"]
    a1 = row["a1"]
    a2 = row["a2"]
    label = row["label"]
    label = "A" if label == "a1" else "B"

    chatml = [
        situation(
            f"Person is debating a concept with AI Assistant."
            f' The AI Assistant is arguing IN FAVOR OF "{sentence}".'
        ),
        thought(
            f'Which of the following candidate sentences support the statement "{sentence}" the best?'
            f"\nChoices:"
            f"\nA): {a1}"
            f"\nB): {a2}"
            '\n'
            f"\nAnswer: {label}"
        ),
    ]

    return dict(chatml=chatml)

In [23]:
dataset = dataset.map(to_chatml).remove_columns(list(set(dataset.column_names) - {"chatml"}))

Map:   0%|          | 0/23128 [00:00<?, ? examples/s]

In [24]:
dataset.push_to_hub("diwank/samantha-debate-argq", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]